Generate initial condition of single localized pulse for LLE continuation

From Oliver Melchert and Ayhan Demircan, pyGLLE: A Python toolkit for solving the generalized Lugiato–Lefever equation. SoftwareX 15 (2021).

https://www.sciencedirect.com/science/article/pii/S235271102100073X
https://github.com/ElsevierSoftwareX/SOFTX_2020_88

In [6]:
# import os module
import os

# import AUTO modules
from auto import AUTOCommands as ac
from auto import AUTOclui as acl
from auto import interactiveBindings as ib
from auto import runAUTO as ra

# root finding, fftb
from scipy.optimize import root, fsolve, newton
import scipy.fftpack as sfft

# import plotting tools
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy.io import savemat

In [ ]:
# find guess for specific set of parameters
theta = 9
P = 9

# find square intensity of background state (CW) corresponding to these parameters
Iout = newton( lambda r : r*( 1 + (theta - r)**2 ) - P**2, np.abs(P/(theta+1j)) )
a = P / (1 + (Iout - theta)**2)
b = (Iout - theta)*P / (1 + (Iout - theta)**2)
print("a, b, norm, square intensity: ", a, b, np.sqrt(Iout), Iout)

In [28]:
# compute single pulse solution to LLE equation

L = 20       # domain length [-L, L]
N = 256      # number of grid points, should be power of 2 since using Fourier

# # initial guess for single pulse, interval [-L, L], no scaling
# x = np.linspace(-L, L, N, endpoint=False)
# k = sfft.fftfreq(x.size,d=x[1]-x[0])*2*np.pi
# coszeta = np.sqrt(8*theta)/(P*np.pi)
# sinzeta = np.sqrt(1-(coszeta**2))
# y = (a + 1j*b) + np.sqrt(2*theta)*( coszeta+1j*sinzeta ) / np.cosh(np.sqrt(theta)*x)
# scale = 1

# compute "guess" for single pulse, scaled for AUTO to interval [0,1]
x = np.linspace(0, 1, N, endpoint=False)
k = sfft.fftfreq(x.size,d=x[1]-x[0])*2*np.pi
coszeta = np.sqrt(8*theta)/(P*np.pi)
sinzeta = np.sqrt(1-(coszeta**2))
xL = 2*L*x - L;
y = (a + 1j*b) + np.sqrt(2*theta)*( coszeta+1j*sinzeta ) / np.cosh(np.sqrt(theta)*xL)
scale = 2*L

# RHS of LLE equation
def LLE(A, P, theta, k, scale):
    return( P -(1+1j*theta)*A + 1j*sfft.fft((-k**2)*sfft.ifft(A))/(scale**2) + 1j*(np.abs(A)**2)*A )

# find root (zero) of LLE equation, starting with initial guess y
# using N=1024 (like in paper), this takes 1000-2000 steps
# using N=256, takes about 125 steps, and produces a result which AUTO can use
# using N=128 does not work for AUTO
yout = root( lambda A: (LLE(A.view(np.complex128),P,theta,k,scale)).view(np.float64),
            np.array(y, dtype=np.complex128).view(np.float64),
            method='krylov', options={'disp': True}).x.view(np.complex128)

0:  |F(x)| = 23.8144; step 1
1:  |F(x)| = 24.6642; step 1
2:  |F(x)| = 8.99539; step 1
3:  |F(x)| = 6.69745; step 1
4:  |F(x)| = 4.88959; step 1
5:  |F(x)| = 4.23189; step 1
6:  |F(x)| = 3.74848; step 1
7:  |F(x)| = 3.50662; step 1
8:  |F(x)| = 2.73268; step 1
9:  |F(x)| = 1.91164; step 1
10:  |F(x)| = 1.41762; step 1
11:  |F(x)| = 1.32292; step 1
12:  |F(x)| = 0.658329; step 1
13:  |F(x)| = 0.457904; step 1
14:  |F(x)| = 0.493425; step 1
15:  |F(x)| = 0.317172; step 1
16:  |F(x)| = 0.19754; step 1
17:  |F(x)| = 0.197178; step 1
18:  |F(x)| = 0.187771; step 1
19:  |F(x)| = 0.155278; step 1
20:  |F(x)| = 0.117314; step 1
21:  |F(x)| = 0.0839406; step 1
22:  |F(x)| = 0.0823881; step 1
23:  |F(x)| = 0.0757936; step 1
24:  |F(x)| = 0.0679786; step 1
25:  |F(x)| = 0.0624618; step 1
26:  |F(x)| = 0.0765764; step 1
27:  |F(x)| = 0.0485369; step 1
28:  |F(x)| = 0.0323662; step 1
29:  |F(x)| = 0.0192961; step 1
30:  |F(x)| = 0.0136646; step 1
31:  |F(x)| = 0.0103116; step 1
32:  |F(x)| = 0.0085

In [29]:
# real and imaginary parts of solution
yr = np.real(yout)
yi = np.imag(yout)

# compute derivatives
yrD = np.real( sfft.fft((1j*k)*sfft.ifft(yr)) / scale )
yiD = np.real( sfft.fft((1j*k)*sfft.ifft(yi)) / scale )

# save for AUTO
np.savetxt('LLE.dat', np.transpose( (x,yr,yi,yrD,yiD) ) ) 

# test with AUTO

# start AUTO with runner object
runner = ra.runAUTO()
# run continuation for a few steps
rp = ac.run(e='LLE', c='LLE.1', runner=runner, NMX=10, PAR={1 : P, 2 : theta})


Starting LLE ...

  BR    PT  TY  LAB    PAR(1)        L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)      MAX U(4)      PAR(3)     
   1     1  EP    1   9.00000E+00   1.85674E+00   1.48236E+00   4.77285E+00   2.44308E+00   9.26925E+00   0.00000E+00
   1     5        2   8.29098E+00   1.78689E+00   1.60799E+00   4.70743E+00   2.80854E+00   9.39941E+00  -2.05251E-09
   1    10  EP    3   8.24329E+00   1.78040E+00   1.61638E+00   4.70008E+00   2.82137E+00   9.37248E+00  -3.96355E-09

 Total Time    0.304E+00
LLE ... done
